<a href="https://colab.research.google.com/github/hwangsaeyeon/dacon/blob/main/lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/mood/train.csv",index_col=0)
test_df = pd.read_csv("/content/drive/MyDrive/mood/test.csv",index_col=0)

In [4]:
train = pd.DataFrame(train_df)
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


## **Null값 전처리**

null값 확인

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

q1~q26 null값 전처리

In [44]:
#train에서 question에 대한 값이 아예 없는 row삭제(8개)
cols = train.columns
train = train.dropna(how='all',subset =cols[0:26])
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


최빈값 구하기

In [45]:
#데이터 인덱스를 nerdiness가 1인 것과 0인것으로 나눔
idx_one = []
idx_zero = []
for i in train.index:
  if train['nerdiness'][i]==1:
    idx_one.append(i)
  elif train['nerdiness'][i]==0:
    idx_zero.append(i)

from pandas._libs.algos import pad
from scipy.stats import mode

cols = list(train.columns)

def mode_val(feature,idx):
  return mode(train[feature][idx])

mode_val_one = []
mode_val_zero = []

#idx_one
for i in cols:
  mode_val_one.append(mode_val(i,idx_one)[0][0])

#idx_zero
for i in cols:
  mode_val_zero.append(mode_val(i,idx_zero)[0][0])

In [51]:
#question null값을 최빈값으로 바꾸기 
col_q = train.columns[0:26]
q_mode_val_zero = mode_val_zero[0:26]
q_mode_val_one = mode_val_one[0:26]
idx=0
for j in col_q:
  for i in train.index:
    if train[j].isna().loc[i] == True:
      if i in idx_one:
        train[j].loc[i] = q_mode_val_one[idx]
      if i in idx_zero:
        train[j].loc[i] = q_mode_val_zero[idx]
  idx+=1

KeyboardInterrupt: ignored

TIPI 값 전처리

In [ ]:
#TIPI값을 최빈값으로 바꾸기 
col_t = train.columns[30:40]
t_mode_val_zero = mode_val_zero[30:40]
t_mode_val_one = mode_val_one[30:40]
idx=0
for j in col_t:
  for i in train.index:
    if train[j].isna().loc[i] == True:
      if i in idx_one:
        train[j].loc[i] = t_mode_val_one[idx]
      if i in idx_zero:
        train[j].loc[i] = t_mode_val_zero[idx]
  idx+=1

In [ ]:
train.info()

engnat null값 전처리 

In [ ]:
engnat1 = ['GBR','CAN','USA','IRL','AUS','SWE','NLD'] #영어가 모국어인 국가
engnat2 = ['DEU','POL','PHL','MYS'] #영어가 모국어가 아닌 국가 

In [ ]:
for i in train.index:
  if train['engnat'].isna()[i] == True:
    if train['country'][i] in engnat1:
      train['engnat'][i] = 1
    elif train['country'][i] in engnat2:
      train['engnat'][i] = 2
    else:
      train['engnat'][i] = 2

In [ ]:
train.info()

Hand, Religion, Orientation, Familysize, Married, ASD null값 전처리

In [ ]:
for i in train.index:
  if train['hand'].isna()[i] == True:
    train['hand'] = 1

for i in train.index:
  if train['religion'].isna()[i] == True:
    train['religion'] = 2

for i in train.index: #400개 널값
  if train['orientation'].isna()[i] == True:
    train['orientation'] = 1

for i in train.index: #400개널값
  if train['familysize'].isna()[i] == True:
    train['familysize'] = 2

for i in train.index:
  if train['married'].isna()[i] == True:
    train['married'] = 1

for i in train.index:
  if train['ASD'].isna()[i] == True:
    train['ASD'] = 2

for i in train.index:
  if train['voted'].isna()[i] == True:
    train['voted'] = 2

for i in train.index:
  if train['gender'].isna()[i] == True:
    train['gender'] = 2

education - 0

In [ ]:
for i in train.index:
  if train['education'].isna()[i] == True:
    train['education'] = 0

In [ ]:
final_train = train.drop('country',axis=1)

## 모델

In [1]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.0 MB/s 
     |████████████████████████████████| 133 kB 39.1 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=0ea1ccfcc3ee525678dab2b729f741fc09ebc96e3482d1380813d7f9cfdefbea
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [108]:
!pip install impyute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [163]:
train = pd.DataFrame(train_df)
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


In [120]:
#train에서 question에 대한 값이 아예 없는 row삭제(8개)
cols = train.columns
train = train.dropna(how='all',subset =cols[0:26])
train

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
index,,,,,,,,,,,,,,,,,,,,,
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1


In [164]:
train = train.drop(['country'],axis=1)

In [17]:
#train_question = train.columns[0:26]
#for i in train_question:   
#  train[i].fillna(value=train[i].mean(), inplace=True)

In [82]:
train.iloc[:,0:10].describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
count,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000
mean,3.964114,4.049160,4.210979,3.779549,3.848786,3.666555,4.164288,3.868663,3.902882,4.093050
std,1.110429,1.195649,1.045993,1.241888,1.206795,1.184701,1.094079,1.315649,1.137548,1.079187
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,4.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,4.000000
50%,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [83]:
train.iloc[:,10:20].describe()

,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
count,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000
mean,3.112793,3.783885,3.693503,3.716649,3.013674,3.427895,3.933498,3.889141,3.375600,3.664955
std,1.577607,1.254005,1.330929,1.170261,1.520329,1.405319,1.148523,1.341142,1.555115,1.184418
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,3.000000,3.000000,1.000000,2.000000,3.000000,3.000000,2.000000,3.000000
50%,3.000000,4.000000,4.000000,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [84]:
train.iloc[:,20:26].describe()

,Q21,Q22,Q23,Q24,Q25,Q26
count,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000
mean,2.962247,2.408018,3.824707,4.281550,3.143877,4.144811
std,1.613651,1.378363,1.373664,0.952707,1.383857,1.061029
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,3.000000,4.000000,2.000000,4.000000
50%,3.000000,2.000000,4.000000,5.000000,3.000000,4.000000
75%,5.000000,3.000000,5.000000,5.000000,4.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [86]:
train.iloc[:,26:29].describe()

,introelapse,testelapse,surveyelapse
count,14992.000000,14992.000000,1.499200e+04
mean,761.892276,388.053162,2.789356e+03
std,13827.724745,8515.300620,1.786431e+05
min,1.000000,25.000000,3.000000e+00
25%,4.000000,82.000000,1.260000e+02
50%,10.000000,106.000000,1.640000e+02
75%,36.000000,140.000000,2.170000e+02
max,855030.000000,474572.000000,1.516699e+07


In [81]:
train.iloc[:,29:39].describe()

,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10
count,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000
mean,2.613927,3.024146,3.447705,3.260872,3.857524,3.666555,3.514141,3.025614,3.148212,2.165888
std,1.159123,1.075908,1.027700,1.192631,0.947197,1.130829,1.038868,1.155572,1.084289,1.043399
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,1.000000
50%,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,3.000000,3.000000,3.000000,2.000000
75%,3.000000,4.000000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,4.000000,3.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [85]:
train.iloc[:,39:55].describe()

,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16
count,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000
mean,0.958645,0.921692,0.529282,0.968450,0.947372,0.101521,0.236059,0.436766,0.067236,0.956043,0.280149,0.210245,0.761273,0.906417,0.959912,0.980590
std,0.199117,0.268665,0.499158,0.174805,0.223297,0.302027,0.424673,0.496002,0.250438,0.205006,0.449087,0.407496,0.426320,0.291258,0.196172,0.137967
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [80]:
train.iloc[:,55:].describe()

,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
count,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000,14992.000000
mean,2.288821,2.163887,1.700840,1.341716,26.745998,1.181297,4.030883,1.798426,1.623733,1.173426,2.685566,1.927161,0.553495
std,0.947611,0.728245,0.556181,0.480865,317.142938,0.485617,3.436636,1.242490,0.496030,0.472659,23.861207,0.281801,0.497147
min,0.000000,0.000000,0.000000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,1.000000,17.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,20.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000
75%,3.000000,3.000000,2.000000,2.000000,27.000000,1.000000,6.000000,2.000000,2.000000,1.000000,3.000000,2.000000,1.000000
max,4.000000,3.000000,3.000000,2.000000,38822.000000,3.000000,12.000000,5.000000,2.000000,3.000000,2919.000000,2.000000,1.000000


In [165]:
train['age'].skew()

122.15677629459286

In [188]:
train['age'].max()
for i in range(len(train)):
  if train['age'][i] >= 90:
    print(i)

1019
1739
4002
4976
6725
9273
12092
14135


In [190]:
train = train.drop(['introelapse','testelapse','surveyelapse','familysize','religion','age'],axis=1)

In [191]:
#null값 변환
from impyute.imputation.cs import mice
np_imputed=mice(train.values) # mice 학습시작
new_train = pd.DataFrame(np_imputed)

# 변환 후 데이터 확인
new_train.head()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,...,1.0,3.0,1.0,20.0,2.0,4.0,2.0,1.0,2.0,1.0
1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,...,2.0,2.0,1.0,49.0,1.0,1.0,1.0,2.0,2.0,1.0
2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,...,1.0,1.0,2.0,43.0,1.0,2.0,2.0,3.0,2.0,1.0
3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,...,3.0,1.0,1.0,17.0,2.0,1.0,2.0,1.0,2.0,1.0
4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,...,2.0,2.0,2.0,18.0,2.0,1.0,2.0,1.0,2.0,0.0
5,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,3.0,...,2.0,1.0,1.0,26.0,1.0,1.0,1.0,1.0,1.0,1.0
6,4.0,3.0,4.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,...,3.0,2.0,2.0,40.0,1.0,1.0,2.0,1.0,2.0,1.0
7,4.0,5.0,4.0,4.0,4.0,4.0,2.0,5.0,3.0,4.0,...,1.0,2.0,2.0,34.0,1.0,5.0,1.0,1.0,2.0,1.0
8,4.0,4.0,3.0,4.0,4.0,5.0,4.0,3.0,3.0,4.0,...,2.0,2.0,1.0,20.0,1.0,1.0,1.0,1.0,2.0,0.0
9,3.0,3.0,4.0,3.0,4.0,2.0,4.0,2.0,4.0,4.0,...,2.0,1.0,2.0,17.0,1.0,1.0,2.0,1.0,2.0,0.0


In [193]:
new_train.to_csv('train_preprocessing.csv',header=train.columns)

In [197]:
train = pd.read_csv('/content/train_preprocessing.csv',index_col=0)

In [198]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(train.iloc[:,0:-1], train.iloc[:,-1])

In [196]:
#kfold 
from numpy import mean, std
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

X = X_resampled 
y = y_resampled

def evaluate_model(X_, y_, model_,thresh):
    cv = RepeatedStratifiedKFold(n_splits=thresh, n_repeats=3, random_state=1)
    scores = cross_val_score(model_, X_, y_, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

thresholds = [5]

for thresh in thresholds:
  rf = RandomForestClassifier(n_estimators=1000)
  scores = evaluate_model(X,y,rf,thresh)
  print(scores.mean())


KeyboardInterrupt: ignored

In [199]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled , test_size = 0.3)

In [521]:
train.columns

Index(['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'TIPI1', 'TIPI2', 'TIPI3', 'TIPI4',
       'TIPI5', 'TIPI6', 'TIPI7', 'TIPI8', 'TIPI9', 'TIPI10', 'VCL1', 'VCL2',
       'VCL3', 'VCL4', 'VCL5', 'VCL6', 'VCL7', 'VCL8', 'VCL9', 'VCL10',
       'VCL11', 'VCL12', 'VCL13', 'VCL14', 'VCL15', 'VCL16', 'education',
       'urban', 'gender', 'engnat', 'hand', 'orientation', 'voted', 'married',
       'familysize', 'ASD', 'nerdiness'],
      dtype='object')

In [200]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics    

rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('정확도 :', metrics.accuracy_score(y_test, y_pred))

정확도 : 0.8058232931726907


In [201]:
import numpy as np
np.random.seed(3)
from lightgbm import LGBMClassifier
import warnings

warnings.filterwarnings('ignore')
lgbm_clf = LGBMClassifier(
            n_estimators=1000,
            learning_rate = 0.1,
            objective = 'binary',
        )
lgbm_clf.fit(X_train, y_train)

y_pred = lgbm_clf.predict(X_test)
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

print(cm)
print(accuracy)

[[1964  557]
 [ 460 1999]]
0.7957831325301205


In [202]:
from xgboost import XGBClassifier
from sklearn import metrics    
model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=4, random_state = 32)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print('정확도 :', metrics.accuracy_score(y_test, y_pred))

정확도 : 0.7829317269076306


In [160]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(lgbm_clf, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Weight,Feature
0.0213 ± 0.0040,Q6
0.0153 ± 0.0042,Q15
0.0127 ± 0.0043,Q1
0.0077 ± 0.0021,Q11
0.0065 ± 0.0033,Q18
0.0062 ± 0.0080,Q12
0.0061 ± 0.0015,TIPI7
0.0058 ± 0.0030,Q19
0.0057 ± 0.0043,Q2
0.0050 ± 0.0027,Q14
